In [1]:
import ipyleaflet
import ipywidgets
import pandas
import numpy

In [2]:
sessions = pandas.read_pickle("../data/real/ChargeSessions_general.pkl")

In [3]:
map_center = [52.179059695, 4.78514509466]
m = ipyleaflet.Map(center = map_center, zoom = 8)

textarea = ipywidgets.Textarea()
textarea.width = '40em'
textarea.height = ''

In [4]:
def clear_map(m):
    current_layers = list(m.layers)[1:]
    for layer in current_layers:
        m.remove_layer(layer)
    layers = []
    
def polygon_generator(offset, size = 1, edge_length_lon = 0.0005, edge_length_lat = 0.0005, shape = 'hexagon'):
    '''
    Generates the coordinates of a hexagon with the center being the offset.

    Args:
        offset: center of the polygon

    Kwargs:
        size: scaling factor for the edge lengths
        edge_length_lon: edge length in longitude direction
        edge_length_lat: edge length in latitude direction
        shape: shape of polygon, e.g. 'hexagon', 'triangle' or 'square'

    Returns:
        An array of coordinates (lon, lat) for the center of the polygon.
    '''

    if shape == 'hexagon':
        stepsize = 60
    elif shape == 'triangle':
        stepsize = 120
    elif shape == 'square':
        stepsize = 90
    else:
        print("Shape undefined in polygon_generator function, now making a hexagon")
        stepsize = 60

    coords = []
    lon, lat = offset
    for angle in range(0, 360, stepsize):
        lon = numpy.cos(numpy.radians(angle)) * edge_length_lon * size + offset[0]
        lat = numpy.sin(numpy.radians(angle)) * edge_length_lat * size + offset[1]
        coords.append([lon, lat])
    return coords

In [5]:
def add_all_css_to_map(m, sessions):    
    clear_map(m)

    features = []
        
    MAX_SIZE = 500
    print(len(sessions['location_key'].unique()))
    for cs in sessions['location_key'].unique():
        coordinates = [sessions[sessions['location_key'] == cs].longitude.values[0],
                       sessions[sessions['location_key'] == cs].latitude.values[0]]

        frequency_of_use = len(sessions[sessions['location_key'] == cs])
        size = 50 + frequency_of_use * MAX_SIZE
        sessions_cs = sessions[sessions['location_key'] == cs].to_json(date_format='iso')

        feature = {"type": "Feature", "properties": {"type": "charging station",
                                                     "cs": cs,
                                                     "frequency_of_use": frequency_of_use,
                                                     "size": size,
                                                     "sessions_cs": sessions_cs,
                                                     "style": {"fillOpacity": 0.8, "smoothFactor": 0, "stroke": True,
                                                               "color": "blue", "fillColor": "blue"}},
                   "geometry": {"type": "Polygon", "coordinates":
                                [polygon_generator(offset = coordinates, size = 0.2, shape = 'square')]}}
        features.append(feature)

    data = {"type": "FeatureCollection", "features": features}

    layer = ipyleaflet.GeoJSON(data = data, hover_style = {'color': 'grey', 'fillColor': 'grey'})

    def click_handler(event=None, id=None, properties=None):
        sessions_cs = pandas.read_json(properties["sessions_cs"])
        sessions_cs['start_connection'] = pandas.to_datetime(sessions_cs['start_connection'])
        sessions_cs['end_connection'] = pandas.to_datetime(sessions_cs['end_connection'])

        sessions_cs = sessions_cs.sort('start_connection')
        first_appearance = sessions_cs['start_connection'].iloc[0].date()
        last_appearance = sessions_cs['start_connection'].iloc[-1].date()

        textarea.value = ("You clicked on a charging station.\n"
                             "Location key(s): \t\t%s\n"
                             "Nr of sessions: \t\t%d\n"
                             "Active between: \t\t%s and %s\n" % (properties['cs'], properties['frequency_of_use'],
                                              first_appearance, last_appearance))

    layer.on_click(click_handler)

    m.add_layer(layer)
    

In [6]:
add_all_css_to_map(m, sessions)  

2540


In [9]:
m

In [8]:
textarea